### Connect to google drive to get the rules file

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Read the rules file and stored each row in pandas dataframe

In [ ]:
import pandas as pd

with open('/content/drive/MyDrive/Colab Notebooks/community.rules') as f_input:
   # store each line in list using list comprehension
    data = [line.strip() for line in f_input]
#Use the result list to create dataframe with rule column that contains row for each rule 
df = pd.DataFrame(data,columns=["Rule"])



### Exploratory Analysis

In [ ]:
# copy the dataframe to do some exploratory analysis
df_rules = df.copy()

In [ ]:
#check the information of the dataframe to see the dataframe columns  
# and columns info such as number of rows ,null values and column data type
df_rules.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4022 entries, 0 to 4021
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Rule    4022 non-null   object
dtypes: object(1)
memory usage: 31.5+ KB


In [ ]:
#check the number of null values in the data 
# we can see the data does not contain any null value
df_rules.isnull().sum()

Rule    0
dtype: int64

In [ ]:
#check the number of duplication
df_rules['Rule'].duplicated().sum()

0

In [ ]:
#check the dataframe columns data types
df_rules.dtypes

Rule    object
dtype: object

## Filter the data 
By:
* Mitre
* Cve
* Nessus
* Ports

#### Create Mitre table and mitre code pivot table

In [ ]:
#T1234
#return rules that contain T followed by 4 digits
# the mitre codes extracted using regex pattren
rslt_df_mitre = df_rules.loc[df['Rule'].str.contains(pat = "T\d{4}",regex=True)] 


In [ ]:
#extract the mitre code using pattren \d{4} here for digits and 4 means 4 digits
rslt_df_mitre['mitre_code']=rslt_df_mitre['Rule'].str.extract("(T\d{4})")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
#show the first 5 rows of mitre table
rslt_df_mitre.head()

,Rule,mitre_code
25,# alert tcp $EXTERNAL_NET any -> $TELNET_SERVE...,T1014
26,# alert tcp $EXTERNAL_NET any -> $TELNET_SERVE...,T1014
27,# alert tcp $EXTERNAL_NET any -> $TELNET_SERVE...,T1014
28,# alert tcp $EXTERNAL_NET any -> $TELNET_SERVE...,T1014
45,# alert tcp $EXTERNAL_NET any -> $HOME_NET 276...,T1078


In [ ]:
print('Number of rules in mitre table',rslt_df_mitre.shape[0])

Number of rules in mitre table 164


In [ ]:
#save the mitre table to excel file
rslt_df_mitre.to_excel('rules_with_miter_code.xlsx',index=False)

In [ ]:
#genarte pivot table to count the total of rules with each mitre codes
pvt_table_mitre = rslt_df_mitre.pivot_table(index='mitre_code',values ='Rule',aggfunc='count')

In [ ]:
pvt_table_mitre.head()

,Rule
mitre_code,
T0000,1
T1001,2
T1014,18
T1018,19
T1020,20


In [ ]:
print('Number of rows in the pivot table',pvt_table_mitre.shape[0])

Number of rows in the pivot table 26


In [ ]:
#save the pivot table into excel file
pvt_table_mitre.to_excel('mitre_pivot_table.xlsx') 

#### Create CVE table and CVE privot table

In [ ]:
# Filter the data to select rows that contain cve code and does not include mitre code then store the result into new dataframe
rslt_df_cve = df_rules.loc[(df_rules['Rule'].str.contains(pat="cve[(,)]\d{4}[-]\d{4,6}",regex=True)) & ~(df_rules['Rule'].str.contains(pat = "T\d{4}",regex=True))] 


In [ ]:
#create new CVE code column
rslt_df_cve['cve_code']=rslt_df_cve['Rule'].str.extract("(cve[(,)]\d{4}[-]\d{4,6})")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
rslt_df_cve.head()

,Rule,cve_code
33,# alert icmp $EXTERNAL_NET any -> $HOME_NET an...,"cve,2000-0138"
34,# alert icmp $EXTERNAL_NET any -> $HOME_NET an...,"cve,2000-0138"
35,# alert udp $EXTERNAL_NET any -> $HOME_NET [31...,"cve,2000-0138"
36,# alert icmp 3.3.3.3/32 any -> $EXTERNAL_NET a...,"cve,2000-0138"
37,# alert icmp $HOME_NET any -> $EXTERNAL_NET an...,"cve,2000-0138"


In [ ]:
print('Number of rows in the rules in CVE table',rslt_df_cve.shape[0])

Number of rows in the rules in CVE table 1528


In [ ]:
#save the 
rslt_df_cve.to_excel('rules_with_cve_cvecode_without_mitre.xlsx',index=False)

In [ ]:
#genarte pivot table to count the total of rules with each CVE codes
pvt_table_cve = rslt_df_cve.pivot_table(index='cve_code',values ='Rule',aggfunc='count')

In [ ]:
pvt_table_cve.head()

,Rule
cve_code,
"cve,1999-0002",3
"cve,1999-0003",5
"cve,1999-0005",3
"cve,1999-0006",2
"cve,1999-0008",1


In [ ]:
print('Number of rows in the pivot table',pvt_table_cve.shape[0])

Number of rows in the pivot table 783


In [ ]:
#save CVE pivot table
pvt_table_cve.to_excel('cve_pivot_table.xlsx') 

#### Nessus table with Nessus code pivot table

In [ ]:
#nessus,12345
df_nessus_without_mitre_cve = df_rules.loc[(df_rules['Rule'].str.contains(pat="nessus[(,)]\d{5}",regex=True))&~(df_rules['Rule'].str.contains(pat="cve[(,)]\d{4}[-]\d{4,6}",regex=True)) & ~(df_rules['Rule'].str.contains(pat = "T\d{4}",regex=True))] 


In [ ]:
#create new nessus_code column for the nessus code that extracted using regex pattren 
df_nessus_without_mitre_cve['nessus_code']=df_nessus_without_mitre_cve['Rule'].str.extract("(nessus[(,)]\d{4})")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
df_nessus_without_mitre_cve.head()

,Rule,nessus_code
4,# alert tcp $HOME_NET any -> $EXTERNAL_NET any...,"nessus,1115"
7,# alert tcp $EXTERNAL_NET 1000:1300 -> $HOME_N...,"nessus,1115"
19,# alert udp $HOME_NET 2140 -> $EXTERNAL_NET an...,"nessus,1005"
64,# alert udp $EXTERNAL_NET any -> $HOME_NET 53 ...,"nessus,1072"
65,# alert tcp $EXTERNAL_NET any -> $HOME_NET 53 ...,"nessus,1002"


In [ ]:
print('Number of rows in the rules in nessuus table',df_nessus_without_mitre_cve.shape[0])

Number of rows in the rules in nessuus table 128


In [ ]:
#save the nessus table to excel file
df_nessus_without_mitre_cve.to_excel('nessus_nesuscode_without_cve_mitre.xlsx',index=False)

In [ ]:
pvt_table_nuesses = df_nessus_without_mitre_cve.pivot_table(index='nessus_code',values ='Rule',aggfunc='count')

In [ ]:
pvt_table_nuesses.head()

,Rule
nessus_code,
"nessus,1002",2
"nessus,1003",1
"nessus,1005",6
"nessus,1007",1
"nessus,1018",1


In [ ]:
#save the nuess pivot table in excel file
pvt_table_nuesses.to_excel('nessus_pivot_table.xlsx')

#### Port table , port number and port without number pivot tables

In [ ]:
#Select the rows that doesnot contian nuess ,cve or mitre codes
df_port_without_nessus_mitre_cve = df_rules.loc[~(df_rules['Rule'].str.contains(pat="nessus[(,)]\d{5}",regex=True))&~(df_rules['Rule'].str.contains(pat="cve[(,)]\d{4}[-]\d{4,6}",regex=True)) & ~(df_rules['Rule'].str.contains(pat = "T\d{4}",regex=True))] 

In [ ]:
df_port_without_nessus_mitre_cve.head()

,Rule,port_search_part,port
0,# alert tcp $HOME_NET 2589 -> $EXTERNAL_NET an...,alert tcp $HOME_NET 2589 -> $EXTERNAL_NET any ...,2589
1,# alert tcp $EXTERNAL_NET any -> $HOME_NET 759...,alert tcp $EXTERNAL_NET any -> $HOME_NET 7597 ...,7597
2,# alert tcp $EXTERNAL_NET any -> $HOME_NET 123...,alert tcp $EXTERNAL_NET any -> $HOME_NET 12345...,12345:12346
3,# alert tcp $HOME_NET 20034 -> $EXTERNAL_NET a...,alert tcp $HOME_NET 20034 -> $EXTERNAL_NET any...,20034
5,# alert tcp $HOME_NET 666 -> $EXTERNAL_NET any...,alert tcp $HOME_NET 666 -> $EXTERNAL_NET any (...,666


In [ ]:
print('Number of rows of the rules in port table',df_port_without_nessus_mitre_cve.shape[0])

Number of rows of the rules in port table 2202


**We follw two steps to extract the port number**


1.   Extract the part of the rule that may contian the port number so when we use the numeric pattern it not return irrelevant numbers.
2.   Extract the port number of the port part of the rule



In [ ]:
#Extract the port part using regex that start with alere and end with msg:
df_port_without_nessus_mitre_cve['port_search_part']= df_port_without_nessus_mitre_cve['Rule'].str.extract("(alert.*\s[($]?msg:)")


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
df_port_without_nessus_mitre_cve['port_search_part'].head()

0    alert tcp $HOME_NET 2589 -> $EXTERNAL_NET any ...
1    alert tcp $EXTERNAL_NET any -> $HOME_NET 7597 ...
2    alert tcp $EXTERNAL_NET any -> $HOME_NET 12345...
3    alert tcp $HOME_NET 20034 -> $EXTERNAL_NET any...
5    alert tcp $HOME_NET 666 -> $EXTERNAL_NET any (...
Name: port_search_part, dtype: object

In [ ]:
#Extract the port from the port part using regex of different digits numbers and digits pattrens to extract single ,range and list of ports
df_port_without_nessus_mitre_cve['port'] = df_port_without_nessus_mitre_cve['port_search_part'].str.extract('(\d{1,6}[,]\d{1,6}[,]\d{1,6}[,]\d{1,6}[,]\d{1,6}[,]\d{1,6}|\d{1,6}[,]\d{1,6}[,]\d{1,6}[,]\d{1,6}[,]\d{1,6}|\d{1,6}[,]\d{1,6}[,]\d{1,6}[,]\d{1,6}|\d{1,6}[,]\d{1,6}[,]\d{1,6}|\d{1,6}[,]\d{1,6}|\d{1,6}[:]\d{1,6}|\d{1,6}[:]|\d{1,6})',expand=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
#Result of the port extraction , 
#Note the majority of rules doesnot contain port number so the return resul is null
df_port_without_nessus_mitre_cve.head()

,Rule,port_search_part,port
0,# alert tcp $HOME_NET 2589 -> $EXTERNAL_NET an...,alert tcp $HOME_NET 2589 -> $EXTERNAL_NET any ...,2589
1,# alert tcp $EXTERNAL_NET any -> $HOME_NET 759...,alert tcp $EXTERNAL_NET any -> $HOME_NET 7597 ...,7597
2,# alert tcp $EXTERNAL_NET any -> $HOME_NET 123...,alert tcp $EXTERNAL_NET any -> $HOME_NET 12345...,12345:12346
3,# alert tcp $HOME_NET 20034 -> $EXTERNAL_NET a...,alert tcp $HOME_NET 20034 -> $EXTERNAL_NET any...,20034
5,# alert tcp $HOME_NET 666 -> $EXTERNAL_NET any...,alert tcp $HOME_NET 666 -> $EXTERNAL_NET any (...,666


In [ ]:
print('Number of rows in the rules in port table',df_port_without_nessus_mitre_cve.shape[0])

Number of rows in the rules in port table 2202


In [ ]:
#save the port table into excel table
df_port_without_nessus_mitre_cve.to_excel('rules_with_port.xlsx',index=False)

In [ ]:
#create pivot table to count each of port or port list in the rules
# exclude rows with null port
pvt_table_port = df_port_without_nessus_mitre_cve[df_port_without_nessus_mitre_cve['port'].notna()].pivot_table(index='port',values ='Rule',aggfunc='count')

In [ ]:
pvt_table_port

,Rule
port,
10011,1
1020,2
1024:,6
1024:65535,1
1025:,5
...,...
9000:,3
9000:9002,1
9100,1


In [ ]:
#save the port pivot table
pvt_table_port.to_excel('port_pivot.xlsx')

In [ ]:
#create dataframe for the port rules without port number
df_ports_without_number = df_port_without_nessus_mitre_cve[df_port_without_nessus_mitre_cve['port'].isnull()]

In [ ]:
print('number of rules that does not contain port number',df_ports_without_number.shape[0])

number of rules that does not contain port number 1678


In [ ]:
#save the rules data without port number 
df_ports_without_number.to_excel('ports_without_numbers.xlsx',index=False)